In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.common import exceptions

import bs4
import json

def bilbasen_interaction(make, model):
    url = f'https://www.bilbasen.dk/brugt/bil/{make}/{model}?IncludeEngrosCVR=true&PriceFrom=0&includeLeasing=false&IncludeCallForPrice=false'
    options = Options()
    options.headless = True
    browser = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
    browser.get(url)
    
    links = []
    next_page = True;
    
    while (next_page):
        anchor = None;
        next_page = False
        
        #Get all a tags
        browser.refresh()
        elmns = browser.find_elements(By.TAG_NAME,"a")
        for elm in elmns:
            
            try:
                #Find the a tags we need to get links
                if(elm.get_attribute("class") == "listing-heading darkLink"):
                    links.append(elm.get_attribute("href"))
                #Check for next page
                if(elm.get_attribute("class") =="next"):
                    print("next")
                    anchor = elm
                    next_page = True
            except exceptions.StaleElementReferenceException:
                print("stale")
        print(len(links))
            #Go to next page
        if (next_page):
            browser.get(anchor.get_attribute("href"))
                
    browser.close()
    return links

In [43]:
links = bilbasen_interaction('vw','up!')

[WDM] - Downloading: 19.0kB [00:00, 8.59MB/s]                   


next
32
next
64
next
stale
96
next
128
next
160
next
192
next
224
next
256
next
stale
288
next
320
next
352
next
stale
384
411


In [18]:
import requests as req
import bs4
import re
import pandas as pd
import json
import time

In [19]:
def get_details(urls):
    df = pd.DataFrame()
    i = 0
    for url in urls:
        i+=1
        pct = i/len(urls)*100
        if(i%50 == 0):
            print(f'{pct}%')
        try:
            info_dict ={}
            html = req.get(url)
            soup = bs4.BeautifulSoup(html.text, 'html.parser')
            scripts = soup.select('script')
            script = scripts[-1].text
            json_string = script.replace("var _props = ","").replace(";","")
            json_object = json.loads(json_string)
            info_dict['price']=json_object['listing']['price']['displayValue']
            info_dict['model_year']=json_object['listing']['vehicle']['modelYear']
            info_dict['km']=json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_history_kilometers_driven']
            info_dict['reg']=json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_history_registration_date']
            info_dict['fuel_type'] = json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_model_fuel_type']
            info_dict['fuel_economy'] = json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_model_fuel_economy']
            info_dict['horse_power'] = json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_model_effect']
            info_dict['gear_type'] = json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_model_gear_type']
            info_dict['zip_code'] = json_object['listing']['seller']['address']['zipCode']
            new_row = pd.Series(data= info_dict)
            df = pd.concat([df,new_row.to_frame().T], ignore_index=True) 
            
        except Exception:
            print(url)
    return df

In [45]:
df = get_details(links)

In [46]:
df

,price,model_year,km,reg,fuel_type,fuel_economy,horse_power,gear_type,zip_code
0,99.000 kr.,2017,39000,2016-09-14,Benzin,24.4,60,M,3650
1,79.900 kr.,2016,115000,2016-04-27,Benzin,24.4,60,M,2880
2,64.800 kr.,2015,148000,2014-07-24,Benzin,24.4,60,M,3400
3,82.900 kr.,2017,111000,2016-12-14,Benzin,24.4,60,M,8300
4,94.800 kr.,2017,138000,2017-03-06,Benzin,22.7,90,M,6760


In [47]:
df.to_csv('../data/up_data.csv', sep=';',index=False)

In [50]:
links[0:1]

['https://www.bilbasen.dk/brugt/bil/vw/up/10-mpi-60-move-up-bmt-5d/5621117']

In [51]:
import json

html = req.get('https://www.bilbasen.dk/brugt/bil/vw/up/10-mpi-60-move-up-bmt-5d/5621117')
soup = bs4.BeautifulSoup(html.text, 'html.parser')
scripts = soup.select('script')
script = scripts[-1].text
json_string = script.replace("var _props = ","").replace(";","")
json_object = json.loads(json_string)
#json_object['tracking']['gtm']['dataLayer']['a']['attr']['vehicle_model_gear_type']
json_object

{'listing': {'externalId': 5621117,
  'syiId': 'e93f58ea-a1be-c6b0-4dab-08da93dfb5f6',
  'tenant': 'bilinfo',
  'canonicalUrl': 'https://www.bilbasen.dk/brugt/bil/vw/up/10-mpi-60-move-up-bmt-5d/5621117',
  'price': {'name': 'Kontantpris', 'displayValue': '99.000 kr.'},
  'vehicle': {'make': 'VW',
   'model': 'Up!',
   'variant': '1,0 MPi 60 Move Up! BMT 5d',
   'modelYear': 2017,
   'ratings': {'average': 4.412079179744731,
    'numberOfReviews': 2140,
    'subRatings': [{'name': 'Køreegenskaber', 'rating': 4.32},
     {'name': 'Driftsomkostninger', 'rating': 4.695744038635677},
     {'name': 'Sikkerhed', 'rating': 4.017840943453281},
     {'name': 'Værdi for pengene', 'rating': 4.614731736889967}]},
   'details': [{'name': 'Modelår', 'displayValue': '2017'},
    {'name': '1. registrering', 'displayValue': '9/2016'},
    {'name': 'Kilometertal', 'displayValue': '39.000 km'},
    {'name': 'Drivmiddel', 'displayValue': 'Benzin'},
    {'name': 'Brændstofforbrug',
     'displayValue': '(NE